"""
This is our oroject mainfile
"""

Modules of python
-----------------

In [271]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from random import randint

Modules with own classes
------------------------

In [272]:
from glassdata import GlassData
from network import NeuralNetwork

---------------------------------------
Data-set on rho and ANN on molar volume
---------------------------------------

In [273]:
# Dataset of rho
filedbrho='DataBase/rho20oxides.csv'
dbrho=GlassData(filedbrho)
dbrho.info()
dbrho.bounds()

**************
Finished loading dataset
Nb of samples: 64421
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 64421
Nb of components: 20


In [274]:
# Determination of the molar volume
dbrho.oxidemolarmass()
dbrho.molarmass()
dbrho.y=dbrho.MolarM/dbrho.y
dbrho.normalize_y()

In [275]:
# Loading of the ANN model
arch=[20,20,20]
nnmolvol=NeuralNetwork(dbrho.noxide,arch,'gelu','linear')
nnmolvol.compile(3.e-4)
nnmolvol.ArchName(arch)
nnmolvol.load('Models/nnmolarvol'+nnmolvol.namearch+'.h5')
nnmolvol.info()

**************


Model: "sequential_181"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_487 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_488 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_489 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_490 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------------------------
Data-set on Young's modulus and ANN on Vt=E/(2G)
------------------------------------------------

In [276]:
filedbE='DataBase/E20oxides.csv'
dbE=GlassData(filedbE)
dbE.info()
dbE.bounds()

**************
Finished loading dataset
Nb of samples: 10846
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 10846
Nb of components: 20


------------------------------
Loading of dissociation energy
------------------------------

In [277]:
datadisso=pd.read_csv('dissociationenergy.csv')
G=np.zeros(dbE.nsample)
for i in range(dbE.nsample):
    G[i]=np.sum(datadisso['G'].values*dbE.x[i,:])
#end for

In [278]:
# Determination of E/G and normalization
dbE.y=dbE.y/(2.*G)
dbE.normalize_y()

------------------------------
Loading of the ANN model on Vt
------------------------------

In [279]:
arch=[20,20,20]
nnmodelEsG=NeuralNetwork(dbE.noxide,arch,'gelu','linear')
nnmodelEsG.compile(1.e-4)
nnmodelEsG.ArchName(arch)
nnmodelEsG.load('Models/nnEsG'+nnmodelEsG.namearch+'.h5')
nnmodelEsG.info()

**************


Model: "sequential_262"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_986 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_987 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_988 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_989 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

---------------------------------------
Data-set on Tannealing=Tg and ANN model
---------------------------------------

In [280]:
# Data-set of Tannealing
filedbTannealing='DataBase/Tannealing20oxides.csv'
dbTannealing=GlassData(filedbTannealing)
dbTannealing.bounds()
dbTannealing.normalize_y()

**************
Finished loading dataset
Nb of samples: 6612
Nb of components: 20


In [281]:
# ANN model on Tannealing
# -----------------------
arch=[20,20,20]
nnTannealing=NeuralNetwork(dbTannealing.noxide,arch,'gelu','linear')
nnTannealing.compile(3.e-4)
nnTannealing.ArchName(arch)
nnTannealing.load('Models/nn'+dbTannealing.nameproperty+nnTannealing.namearch+'.h5')
nnTannealing.info()

**************


Model: "sequential_187"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_585 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_586 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_587 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_588 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Data-set on Tmelt
-----------------

In [282]:
filedbTmelt='DataBase/Tmelt19oxides.csv'
dbTmelt=GlassData(filedbTmelt)
dbTmelt.info()
dbTmelt.bounds()
dbTmelt.normalize_y()

**************
Finished loading dataset
Nb of samples: 5996
Nb of components: 19
**************
Finished loading dataset
Nb of samples: 5996
Nb of components: 19


In [283]:
# ANN model on Tmelt
# ------------------
arch=[20,20,20]
nnTmelt=NeuralNetwork(dbTmelt.noxide,arch,'gelu','linear')
nnTmelt.compile(3.e-4)
nnTmelt.ArchName(arch)
nnTmelt.load('Models/nn'+dbTmelt.nameproperty+nnTmelt.namearch+'.h5')
nnTmelt.info()

**************


Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_116 (Dense)               │ (None, 20)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,263 (4.94 KB)

 Trainable params: 1,261 (4.93 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------
Data-set on Tliq and ANN model
------------------------------

In [284]:
filedbTliq='DataBase\Tsoft20oxides.csv'
dbTliq=GlassData(filedbTliq)
dbTliq.info()
dbTliq.bounds()
dbTliq.normalize_y()

**************
Finished loading dataset
Nb of samples: 12531
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 12531
Nb of components: 20


<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\vivgh\AppData\Local\Temp\ipykernel_33700\4115716478.py:1: SyntaxWarning: invalid escape sequence '\T'
  filedbTliq='DataBase\Tsoft20oxides.csv'


In [285]:
# ANN model on Tliq
# -----------------
arch=[32,32,32,32]
nnTliq=NeuralNetwork(dbTliq.noxide,arch,'gelu','linear')
nnTliq.compile(3.e-4)
nnTliq.ArchName(arch)
#modelfile='Models\nn'+dbTliq.nameproperty+nnTliq.namearch+'.h5'
modelfile='Models/nnTsoft3c20.h5'
nnTliq.load(modelfile)
nnTliq.info()

**************


Model: "sequential_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_172 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_173 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_174 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_175 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------------------
Determination of the bounds for each oxide
------------------------------------------

# Algo genetique

## Variables utiles

In [286]:
labels = dbrho.oxide
N_oxides = len(labels)
available_mat = ['SiO2', 'Al2O3', 'MgO', 'CaO', 'Na2O', 'K2O','ZnO', 'TiO2']
prop_label = ['rho','E','Tg','Tmelt']
columns = list(labels)+prop_label+['F']

#Contraintes
xmaxt=np.array([dbrho.xmax,dbE.xmax,dbTannealing.xmax,np.append(dbTmelt.xmax,1.),dbTliq.xmax])
xmax=np.zeros(dbrho.noxide)
for i in range(dbrho.noxide):
    if dbrho.oxide[i] in available_mat:
        xmax[i]=np.min(xmaxt[:,i])

xmin = np.zeros(dbrho.noxide)
xmin[list(dbrho.oxide).index('SiO2')] = 0.35
xmin[list(dbrho.oxide).index('Na2O')] = 0.1

#Parametres fitness function
weight=[0,0.3,0,0.7]
minimize=[True,False,False,True]

N_generations = 10
N_population = 100000

survivor_rate = 0.5
parent_rate = 0.2
child_rate = 0.4
mutation_rate = 0
immigration_rate = 1 - survivor_rate - child_rate

#population = np.zeros((N_population,len(labels) + len(prop_label) + 1))

N_parents = int(parent_rate * N_population)
N_childs = int(child_rate * N_population)
N_mutants = int(0.1 * N_population)

epsilon = 0.05


## Creation de generations

In [287]:
def init_pop(N_population):
    xglass,_=dbrho.better_random_composition(N_population,xmin,xmax)
    return xglass

In [288]:
def prop_calculation(population):
    rho=dbrho.GlassDensity(nnmolvol,dbrho.oxide,population)
    E=dbE.YoungModulus(nnmodelEsG,datadisso,dbE.oxide,population)
    Tg=dbTannealing.physicaly(nnTannealing.model.predict(population).transpose()[0,:])
    Tmelt=dbTmelt.physicaly(nnTmelt.model.predict(population[:,:-1]).transpose()[0,:])
    return np.vstack((rho,E,Tg,Tmelt)).transpose()

In [289]:
def normalize(prop):
    return (prop - prop.min(axis=0))/(prop.max(axis=0)-prop.min(axis=0))

In [290]:
#prop est une array avec les proprietes du verre normalisées, weight est le poids qu'on accorde
#à chacune des proprietes, et minize est une liste de booléens selon qu'on veuille minimiser
#ou maximiser une certaine variable
def fitness_func(prop_normalized,weight,minimize):
    rating = np.zeros(prop_normalized.shape[0])
    for i in range(len(weight)):
        if minimize[i]:
            rating += (1-prop_normalized[:,i])*weight[i]
        else:
            rating += prop_normalized[:,i]*weight[i]
    return rating

In [291]:
# Trie la population par F decroissant et renvoie cette population triée avec une nuovelle colonne qui represente 
# le fitness de chaque composition.
def sort_by_f(population,properties,F):
    population_info = np.column_stack((population,properties,F))
    sorted_arr = population_info[population_info[:, -1].argsort()][::-1]
    return sorted_arr

In [292]:
def compute(population):
    print(population.shape)
    prop = prop_calculation(population)
    normalized_prop = normalize(prop)
    F = fitness_func(normalized_prop,weight,minimize)
    sorted_arr = sort_by_f(population, prop, F)
    return sorted_arr

In [293]:
population = init_pop(N_population)

In [294]:
def population_selection(sorted_population):
    survivors = sorted_population[:int(N_population*survivor_rate)]
    parents = sorted_population[:N_parents]
    #to_be_mutated = sorted_population[int(N_population*survivor_rate):int(N_population*survivor_rate)+int(N_population*mutation_rate)]
    return survivors,parents

In [295]:
def crossover (parents) :
    #Dans parents chaque individu est représenté par 21 floats dont le dernier est la valeur de fitness
    childs = np.array([[0.] * len(parents[0])] * N_childs)
    for i in range (N_childs) :
        i1 = randint(0, N_parents-1)
        i2 = randint(0, N_parents-2)
        if i2 == i1 : #problème si deux fois le même parent !!
            i2 += 1
        w1 = parents[i1, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 1
        w2 = parents[i2, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 2
        childs[i] = (w1 * parents[i1] + w2 * parents[i2]) #moyenne pondérée
    return (childs)

In [296]:
def mutation (mutants) :
    #les mutants sont les meilleures compositions entre 40% et 50%
    for j in range (N_mutants) :
        iplus = randint(0, 19) #choix de l'oxyde qui gagne epsilon
        imoins = randint(0, 19) #choix de l'oxyde qui perd epsilon
        if imoins == iplus : #problème si deux fois le même oxyde !!
            imoins = (1 + imoins)%19
        mutantPlus = mutants[j, iplus]
        mutantMoins = mutants[j, imoins]
        if (mutantMoins > epsilon + xmin[j]) and (mutantPlus < xmax[j] - epsilon) :
            mutants[j, iplus] = mutantPlus + epsilon
            mutants[j, imoins] = mutantMoins - epsilon
    return (mutants)

In [297]:
def new_population(sorted_population):
    survivors,parents = population_selection(sorted_population)
    child = crossover(parents)
    immigrants = init_pop(N_population- (len(survivors) + len(child)))
    new_population = np.vstack((np.vstack((survivors,child))[:,:N_oxides],immigrants))
    return new_population

In [298]:
def evolution(prev_generation,N):
    next_generation = prev_generation
    for i in range(N-1):
        next_generation = compute(new_population(compute(next_generation[:,:20]))[:,:20])
        if i%5 == 4:
            pd.DataFrame(compositions,columns=columns).to_csv(f'generation_{i}.csv')
    next_generation = compute(new_population(compute(next_generation[:,:20]))[:,:20])
    return next_generation

In [299]:
initial_pop = init_pop(N_population)
compositions = evolution(initial_pop,N_generations)

(100000, 20)
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 908us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 690us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 674us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 740us/step
(100000, 20)
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 643us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 830us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 898us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 892us/step
(100000, 20)
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 915us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 771us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 922us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 859us/step
(100000, 20)
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 844us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 670us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 966us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 667us/step
(100000, 20)
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 779us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 628us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 983us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 800us/step
(100000, 20)
3125/3125 ━━━━━━━━━━━━

In [300]:
df = pd.DataFrame(compositions,columns=columns)

In [301]:
df.to_csv('generation_final.csv')

In [302]:
df

,SiO2,B2O3,Al2O3,MgO,CaO,BaO,Li2O,Na2O,K2O,ZnO,...,TiO2,GeO2,ZrO2,P2O5,V2O5,rho,E,Tg,Tmelt,F
0,0.366177,0.0,0.000141,0.162449,0.014187,0.0,0.0,0.213707,0.189793,0.025290,...,0.028255,0.0,0.0,0.0,0.0,2740.667536,62.331922,656.943097,933.063855,0.770266
1,0.391972,0.0,0.025941,0.155401,0.026281,0.0,0.0,0.189129,0.181878,0.028852,...,0.000545,0.0,0.0,0.0,0.0,2699.498264,80.462456,704.509209,1090.814148,0.734952
2,0.368893,0.0,0.016211,0.147966,0.020417,0.0,0.0,0.296594,0.141069,0.001043,...,0.007808,0.0,0.0,0.0,0.0,2675.474973,62.540691,605.470029,993.536696,0.732903
3,0.354071,0.0,0.044236,0.061856,0.027788,0.0,0.0,0.136973,0.003495,0.302049,...,0.069532,0.0,0.0,0.0,0.0,3194.697769,93.475446,818.361628,1194.561946,0.715575
4,0.377997,0.0,0.007631,0.108598,0.011514,0.0,0.0,0.125996,0.017283,0.318812,...,0.032168,0.0,0.0,0.0,0.0,3200.808833,102.429383,822.952917,1252.601528,0.710652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.682676,0.0,0.062399,0.021676,0.031527,0.0,0.0,0.130331,0.040520,0.008093,...,0.022779,0.0,0.0,0.0,0.0,2506.651529,72.098150,824.573887,1875.676444,0.210886
99996,0.617194,0.0,0.105817,0.048831,0.000849,0.0,0.0,0.128652,0.038785,0.008898,...,0.050973,0.0,0.0,0.0,0.0,2528.632011,75.962581,890.442894,1906.234020,0.205290
99997,0.663449,0.0,0.080571,0.103530,0.004823,0.0,0.0,0.102682,0.018503,0.017731,...,0.008711,0.0,0.0,0.0,0.0,2503.740883,76.840705,923.910004,1914.710192,0.203053
99998,0.616675,0.0,0.125662,0.011146,0.045791,0.0,0.0,0.122934,0.077137,0.000124,...,0.000532,0.0,0.0,0.0,0.0,2496.976548,72.249979,863.214801,1912.297619,0.188349
